# Road Following - Build TensorRT model for live demo

In this notebook, we will optimize the model we trained using TensorRT.

## Load the trained model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebook's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [ ]:
import torchvision
import torch
from jetbot.utils import model_selection
import os 
import ipywidgets.widgets as widgets
from ipywidgets.widgets import Box, HBox, VBox, Layout, Label
import traitlets

# The path of trt models is 'MODEL_REPO_DIR_DOCKER' which is set in /jetbot/utils/model_selection.py,
# which may be modified if you change the file path of trt models, 'MODEL_REPO_DIR_DOCKER' or dir_model_repo.

dir_model_repo = os.environ['MODEL_REPO_DIR_DOCKER']
print(dir_model_repo)

# pth_ms = model_selection(core_library = "Pytorch")  # if 'MODEL_REPO_DIR_DOCKER' is used.
pth_ms = model_selection(core_library = "Pytorch", dir_model_repo=dir_model_repo)

In [ ]:
pth_ms.model_function = "classifier"

model_type_widget = widgets.Select(options=pth_ms.model_type_list, value=pth_ms.model_type_list[0],
                                      description='Model Type:')
traitlets.dlink((pth_ms, 'model_type_list'), (model_type_widget, 'options'))
traitlets.dlink((model_type_widget, 'value'), (pth_ms, 'model_type'))

model_path_widget = widgets.Select(options=pth_ms.model_path_list, description='Model Path:',
                                      layout=Layout(width='65%'))
traitlets.dlink((pth_ms, 'model_path_list'), (model_path_widget, 'options'))
traitlets.dlink((model_path_widget, 'value'), (pth_ms, 'model_path'))

display(HBox([model_type_widget, model_path_widget]))


In [ ]:
# load pytorch model with no pretrained parameters
from jetbot.utils.model_selection import load_tune_pth_model
model_path = model_path_widget.value
pth_model_name = model_path_widget.value.split('/')[-1].split('.')[0].split('_', 4)[-1]
print(pth_model_name, '\n', model_path)

model, model_type = load_tune_pth_model(pth_model_name=pth_model_name, pretrained=False)
model = model.cuda().eval().half()


Next, load the trained weights from the model_path (e.g.``best_steering_model_xy_<<pth_model_name>>.pth``) file that you uploaded.

In [ ]:
model.load_state_dict(torch.load(model_path))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [ ]:
device = torch.device('cuda')

## TensorRT

> Note: if you are running with docker container, you may not need to do the following installation. 

> If your setup does not have `torch2trt` installed, you need to first install `torch2trt` by executing the following in the console.
```bash
cd $HOME
git clone https://github.com/NVIDIA-AI-IOT/torch2trt
cd torch2trt
sudo python3 setup.py install
```

Convert and optimize the model using torch2trt for faster inference with TensorRT. Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete.

In [ ]:
from torch2trt import torch2trt

if pth_model_name == 'inception_v3':
    data = torch.zeros((1, 3, 299, 299)).cuda().half()   # inception_v3
else:
    data = torch.zeros((1, 3, 224, 224)).cuda().half()  # resnet

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
import pandas as pd

path_trt_model = os.path.join(dir_model_repo, 'road_following', "best_steering_model_xy_trt_"+pth_model_name+'.pth')
torch.save(model_trt.state_dict(), path_trt_model)

df_file = os.path.join(dir_model_repo, 'trt_model_tbl.csv')
if os.path.isfile(df_file):
    df = pd.read_csv(df_file, header=None)
else:
    df = pd.DataFrame()

trt_model_path_tbl = os.path.join('.', 'road_following', "best_steering_model_xy_trt_"+pth_model_name+'.pth')
df = df.append([["classifier", model_type, trt_model_path_tbl]], ignore_index = False)
df = df.drop_duplicates()
df.to_csv(df_file, header=False, index=False)


## Next
Open live_demo_trt.ipynb to move JetBot with the TensorRT optimized model.